In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-op"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
openjdk-8-jdk-headless is already the newest version (8u362-ga-0ubuntu1~20.04.1).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd



In [ ]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [ ]:
# YOUR
txt = spark.read.text('/content/drive/MyDrive/soc-LiveJournal1Adj.txt')


In [ ]:
import re, sys
from pyspark import SparkConf, SparkContext





In [ ]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

lines = sc.textFile("/content/drive/MyDrive/soc-LiveJournal1Adj.txt")

def lines2cands(l):
    uvs = re.split(r'[,\t]', l.strip()) # 'u' 'a' 'b' 'c' 'd'
    uvs = list(map(int, uvs)) # u a b c d
    u = uvs[0]
    vs = sorted(uvs[1:])
    # each cand pair has score 1
    ret = []
    for i in range(len(vs)):
        for j in range(i):
            ret.append(((vs[j], vs[i]), 1))
    # if already connected, score -inf
    minf = -float('inf')
    for v in vs:
        if u < v:
            ret.append(((u, v), minf))
        else:
            ret.append(((v, u), minf))
    # returns [] if no friend. flatMap ignores []
    return ret

cands = lines.flatMap(lines2cands)

cnt = cands.reduceByKey(lambda x, y: x + y) # ((a b) num)
cnt = cnt.filter(lambda kv: kv[1] > 0) # filter connected
# up to here, those w/o 2nd deg friends are excluded

def cnt2suggest(uv_num):
    uv, num = uv_num
    u, v = uv
    return (u, (num, v)), (v, (num, u))

suggest = cnt.flatMap(cnt2suggest) # (a, (num b)) (b, (num a))
suggest = suggest.groupByKey() # (a, ((num b) (num c)) )

def processSuggest(suggest):
    u, vs = suggest
    # sort by number of common friends
    vs = sorted(vs, key=lambda v: (-v[0], v[1]))
    vs = vs[:10]
    # remove numbers. leave friend ids
    vs = list(zip(*vs))[1]
    return u, vs

suggest = suggest.map(processSuggest)

suggest = suggest.sortBy(lambda u_vs: u_vs[0])




In [ ]:
suggest.saveAsTextFile('bai1.homework01')

sc.stop()
